In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import io
import os

# Load Parquet file into a DataFrame
df = pd.read_parquet("../data/processed_sticker_dataset.parquet")
print(df.head())

                                  combined_embedding  \
0  [0.05615041, 0.06784809, -0.03342954, 0.037553...   
1  [-0.124234326, 0.07463956, -0.011985385, 0.004...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [-0.06495428, -0.04292713, 0.013164402, 0.0220...   
4  [0.027918205, 0.075559475, 0.03622711, -0.0181...   

                                          image_path  
0  ../data/tensor_images/AlexatorStickers\cartoon...  
1  ../data/tensor_images/AlexatorStickers\cartoon...  
2  ../data/tensor_images/AlexatorStickers\cartoon...  
3  ../data/tensor_images/AlexatorStickers\cartoon...  
4  ../data/tensor_images/AlexatorStickers\cartoon...  


In [ ]:
import torch
from diffusers import DiffusionPipeline

# Load the smallest LDM model
pipe = DiffusionPipeline.from_pretrained("CompVis/ldm-text2im-large-256", torch_dtype=torch.float32)

# Move model to CPU
pipe.to("cpu")

print("Model running on:", pipe.device)

for index, row in df.iterrows():
    # Extract the image path from the Parquet file (local file path)
    image_path = row['image_path']
    captions = row['comined_embedding'].tolist()

    # 4. Check if the image path is a tensor file (.pt)
    if image_path.endswith(".pt"):
        try:
            # Load the tensor from the .pt file
            tensor_image = torch.load(image_path)

            # Ensure the tensor is in the correct shape (e.g., C x H x W)
            if tensor_image.ndimension() == 3:  # Check if it's a 3D tensor (C, H, W)
                # Convert the tensor to a PIL image (assuming the tensor is in the range [0, 1])
                tensor_image = tensor_image.permute(1, 2, 0)  # Convert from C x H x W to H x W x C
                tensor_image = tensor_image.numpy()  # Convert to numpy array
                pil_image = Image.fromarray((tensor_image * 255).astype('uint8'))  # Convert to uint8 for PIL
            else:
                print(f"Warning: Tensor shape of image '{image_path}' is not valid. Skipping.")
                continue

        except Exception as e:
            print(f"Error loading tensor image from {image_path}: {e}")
            continue

    else:
        # If the image path is not a tensor file, assume it's a normal image file
        if os.path.exists(image_path):
            pil_image = Image.open(image_path)
        else:
            print(f"Warning: Image path '{image_path}' does not exist. Skipping.")
            continue

prompt = "unicorn wearing suit"
image = pipe(prompt, guidance_scale=7.5).images[0]

# Save & Show Image
image.save("cpu_generated_sticker.png")
image.show()



KeyboardInterrupt: 

In [ ]:
from PIL import Image
from transformers import CLIPTokenizer
import torch

# Load the CLIP tokenizer (used for text processing in Stable Diffusion)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# Preprocess the images: Resize and normalize to (512, 512)
def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        try:
            img = Image.open(image_path).convert("RGB")
            img = img.resize((512, 512))  # Resize image to 512x512
            images.append(img)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            continue
    return images

# Tokenize the captions
def tokenize_captions(captions):
    return tokenizer(captions, padding=True, truncation=True, return_tensors="pt")

images = preprocess_images(image_paths)
caption_tokens = tokenize_captions(captions)
